In [3]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [12]:
def process_image(image):
    # TODO: put your pipeline here,

    # define the x and y size 
    ysize = image.shape[0]
    xsize = image.shape[1]

    # make a copy file of the original image
    color_select = np.copy(image)
    # grayscale the copy file
    gray = cv2.cvtColor(color_select, cv2.COLOR_RGB2GRAY)

    # selecting the religion to identify lanelines
    vertices = np.array([[(0.1*xsize,ysize),(0.45*xsize,0.6*ysize),(0.6*xsize,0.6*ysize),(xsize,ysize)]],dtype=np.int32)

   
    def region_of_interest(img, vertices):
       
        # defining a blank mask to start with
        mask = np.zeros_like(img)   

        # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
        if len(img.shape) > 2:
            channel_count = img.shape[2]  
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255

        # filling pixels inside the polygon defined by "vertices" with the fill color    
        cv2.fillPoly(mask, vertices, ignore_mask_color)

        #returning the image only where mask pixels are nonzero
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image
    
    # define a kernel size for Gaussian smoothing 
    kernel_size = 3
    blur_gray = cv2.GaussianBlur(gray,(kernel_size,kernel_size),0)

    # setting up the thresholds for Canny edge detection
    low_threshold = 70
    high_thresold = 210
    edges = cv2.Canny(blur_gray,low_threshold,high_thresold)

    # decide on the area of the given picture
    masked_edges = region_of_interest(edges,vertices)
    
    #define the Hough transfrom pareameters 

    rho = 1
    theta = np.pi/180
    threshold = 10
    min_line_length = 40
    max_line_gap = 100
    line_image = np.copy(image)* 0

    # Run Hough on edge detected image
    lines = cv2.HoughLinesP(masked_edges,rho,theta,threshold,np.array([]),
                           min_line_length,max_line_gap)
    
    #Iterate over the output"lines" and draw lines on the blank
    color = (255,0,0)
    thickness = 10
   
    # define the cordinates of the lines 
    # and also gradients of the left line and right line
    left_x,left_y, right_x,right_y,grad_list_left,grad_list_right = [],[],[],[],[],[]
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 <= xsize/2.:
                left_x.append(x1)
                left_y.append(y1)
            else:
                right_x.append(x1)
                right_y.append(y1)
            if x2 <= xsize/2.:
                left_x.append(x2)
                left_y.append(y2)
            else:
                right_x.append(x2)
                right_y.append(y2)
                               
            # when gradient is positive, append to the list of left line, otherwise to the right 
            if(((y2-y1) / (x2-x1)) < 0):
                grad_list_left.append((y2-y1) / (x2-x1))
            else:
                grad_list_right.append((y2-y1) / (x2-x1))
                
    # taking a mean value of gradients
    mean_grad_left = np.mean(grad_list_left)
    mean_grad_right = np.mean(grad_list_right) 

    # extend the left and right lines to the bottom edge of the given picture
    # and define the x cordinates 
    p_x = int(((ysize - max(left_y)) + min(left_x) * mean_grad_left) / mean_grad_left)
    m_x = int(((ysize - max(right_y)) + max(right_x) * mean_grad_right) / mean_grad_right)

    # draw the left and right lines on the blank images
    cv2.line(line_image, (p_x, ysize), (max(left_x), min(left_y)), color, thickness)
    cv2.line(line_image, (min(right_x),min(right_y)), (m_x, ysize), color, thickness)
             
    # Draw the created lines on the original image
    result = cv2.addWeighted(image,1,line_image,0.5,0)
   
    return result

In [13]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4



100%|█████████▉| 221/222 [00:13<00:00, 15.84it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 3.04 s, sys: 223 ms, total: 3.26 s
Wall time: 15.7 s


In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [14]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
yellow_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)


[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4



 50%|████▉     | 338/682 [00:22<00:24, 13.98it/s]


 95%|█████████▍| 646/682 [00:45<00:02, 13.53it/s]


100%|█████████▉| 681/682 [00:47<00:00, 14.27it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 9.45 s, sys: 744 ms, total: 10.2 s
Wall time: 49.5 s


In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [17]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/challenge.mp4")
challenge_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False)


[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4



  2%|▏         | 4/251 [00:00<00:13, 17.67it/s]

ValueError: cannot convert float NaN to integer